In [24]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


 # Memory hierarchy

## How much time did I spent on "memory"

Remember our demo long time ago?

In [ ]:
compare([do_render_code("./madd/madd_A.c", lang="c++", show=["//START", "//END"]),do_render_code("./madd/madd_B.c", lang="c++", show=["//START", "//END"])])

In [ ]:
render_code("./mv/matvec.c", lang="c++", show=["//I_FIRST_START", "//I_FIRST_END"])

In [ ]:
! cd mv; make clean; make; ./matvec 16 1 1 >> mv_addresses.csv ; head -n 100 mv_addresses.csv

In [ ]:
! cd mv; ./matvec 16 1 0 >> mv_addresses_j_first.csv ; head -n 100 mv_addresses_j_first.csv

## The memory hierarchy of my computer

In [19]:
# Hung-Wei's Desktop
! lscpu | grep "Model name"
! getconf -a | grep CACHE

Model name:                      13th Gen Intel(R) Core(TM) i7-13700
LEVEL1_ICACHE_SIZE                 32768
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 49152
LEVEL1_DCACHE_ASSOC                12
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  2097152
LEVEL2_CACHE_ASSOC                 16
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  31457280
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [20]:
# Your CS203 Cluster
! cs203 job run "lscpu | grep 'Model name'; getconf -a | grep CACHE"

htseng
srun -N1 -p datahub lscpu | grep 'Model name'
Model name:                         12th Gen Intel(R) Core(TM) i3-12100F
htseng
srun -N1 -p datahub  getconf -a | grep CACHE
LEVEL1_ICACHE_SIZE                 32768
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 49152
LEVEL1_DCACHE_ASSOC                12
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  1310720
LEVEL2_CACHE_ASSOC                 10
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  12582912
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [21]:
# Jetson Nano (NV Tegra X1)
! ssh htseng@nano-2 "lscpu | grep 'Model name'; getconf -a | grep CACHE"

Model name:          Cortex-A57
LEVEL1_ICACHE_SIZE                 0
LEVEL1_ICACHE_ASSOC                0
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 0
LEVEL1_DCACHE_ASSOC                0
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  0
LEVEL2_CACHE_ASSOC                 0
LEVEL2_CACHE_LINESIZE              0
LEVEL3_CACHE_SIZE                  0
LEVEL3_CACHE_ASSOC                 0
LEVEL3_CACHE_LINESIZE              0
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


## How blocks are stored in a cache?

How these address are stored "if" they're in a direct-mapped, 16B-sized blocks, 16-block cache?

In [ ]:
! cd mv; make clean; make; ./matvec 16 1 

In [ ]:
!echo "element,address"> addresses.csv; ./mv/matvec 16 1 >> addresses.csv
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
index_bits = int(math.log2(number_of_blocks))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

What if we have a 2-way, 16-byte blocked, 16-block cache?

In [ ]:
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
way_assoc=2
number_of_sets = number_of_blocks/way_assoc
index_bits = int(math.log2(number_of_sets))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

## Cache performance of code on "real machines"

### NVIDIA Jetson Nano -- Tegra X1

In [23]:
render_code("4way_madd/madd.c", show=["//START","//END"])

// 4way_madd/madd.c:51-63 (13 lines)
  //START
    for(i = 0; i < RUN_ARRAY_SIZE; i++)
    {
        e[i] = (a[i] * b[i] + c[i])/d[i];
#ifdef DUMP
        fprintf(stderr, "a[%d], %p\n",i,&a[i]);
        fprintf(stderr, "b[%d], %p\n",i,&b[i]);
        fprintf(stderr, "c[%d], %p\n",i,&c[i]);
        fprintf(stderr, "d[%d], %p\n",i,&d[i]);
        fprintf(stderr, "e[%d], %p\n",i,&e[i]);
#endif
    }
  //END

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

#### Run without the 4-way matrix add loop code.

In [28]:
# Run it "without" the above code.
! ssh htseng@nano-2 "lscpu; cd courses/CS203/demo/memory/4way_madd/; make clean madd_nano; valgrind --tool=cachegrind ./madd_nano 16384 0 "

Architecture:        aarch64
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  1
Core(s) per socket:  4
Socket(s):           1
Vendor ID:           ARM
Model:               1
Model name:          Cortex-A57
Stepping:            r1p1
CPU max MHz:         1479.0000
CPU min MHz:         102.0000
BogoMIPS:            38.40
L1d cache:           32K
L1i cache:           48K
L2 cache:            2048K
Flags:               fp asimd evtstrm aes pmull sha1 sha2 crc32
rm -f madd_intel madd_nano *_O3 *~ madd_A_fission cachegrind* *.perf madd_dump
cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g -DNANO madd.c -o madd_nano
==9861== Cachegrind, a cache and branch-prediction profiler
==9861== Copyright (C) 2002-2017, and GNU GPL'd, by Nicholas Nethercote et al.
==9861== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==9861== Command: ./madd_nano 16384 0
==9861== 
--9861-- Warning: Cannot auto-detect cache config, using defaults.
--9861--    

Too much detail! Let's use grep to narrow down the outputs.

In [44]:
# Run it "without" the above code.
! ssh htseng@nano-2 "cd courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_nano 16384 0 >& nano_without_loop.perf; grep 'D   refs\|D1' nano_without_loop.perf"

==17216== D   refs:      3,002,342  (1,842,830 rd   + 1,159,512 wr)
==17216== D1  misses:       13,658  (    2,738 rd   +    10,920 wr)
==17216== D1  miss rate:       0.5% (      0.1%     +       0.9%  )


Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [45]:
! ssh htseng@nano-2 "cd courses/CS203/demo/memory/4way_madd/;valgrind --tool=cachegrind ./madd_nano 16384 8192 >& nano_with_loop.perf; grep 'D   refs\|D1' nano_with_loop.perf"

==17271== D   refs:      3,022,826  (1,859,214 rd   + 1,163,612 wr)
==17271== D1  misses:       34,151  (   19,132 rd   +    15,019 wr)
==17271== D1  miss rate:       1.1% (      1.0%     +       1.3%  )


In [46]:
# Let's do some math here
total_number_of_accesses_before_the_loop =     3002342
total_number_of_accesses_after_the_loop =  3022826
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop =  13658
total_number_of_misses_after_the_loop = 34151
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 20484 misses in the loop 20493 miss_rate 1.000439


In [ ]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 32768
B = 64
A = 4
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

### Intel Core i7 13700 -- 12-way L1, 64B-blocked, 48KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [36]:
! cd 4way_madd; make clean; make EXTRA_CFLAGS="-mno-avx -O4" madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

make: 'madd_dump' is up to date.
60.000000


,element,address,tag,index
0,a[0],0x7fe82b2af000,0x7fe82b2af,0x0
1,b[0],0x7fe82b2bf000,0x7fe82b2bf,0x0
2,c[0],0x7fe82b2cf000,0x7fe82b2cf,0x0
3,d[0],0x7fe82b2df000,0x7fe82b2df,0x0
4,e[0],0x7fe82b2ef000,0x7fe82b2ef,0x0
5,a[1],0x7fe82b2af008,0x7fe82b2af,0x0
6,b[1],0x7fe82b2bf008,0x7fe82b2bf,0x0
7,c[1],0x7fe82b2cf008,0x7fe82b2cf,0x0
8,d[1],0x7fe82b2df008,0x7fe82b2df,0x0
9,e[1],0x7fe82b2ef008,0x7fe82b2ef,0x0


#### Run without the 4-way matrix add loop code.

In [39]:
# Run it "without" the above code.
! lscpu; cd ~/courses/CS203/demo/memory/4way_madd/; rm madd_intel; make EXTRA_CFLAGS="-mno-avx -O4" madd_intel; valgrind --tool=cachegrind ./madd_intel 16384 0 >& intel_without_loop.perf; grep 'D   refs\|D1' intel_without_loop.perf

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          24
On-line CPU(s) list:             0-23
Thread(s) per core:              1
Core(s) per socket:              16
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           183
Model name:                      13th Gen Intel(R) Core(TM) i7-13700
Stepping:                        1
CPU MHz:                         5095.939
CPU max MHz:                     5200.0000
CPU min MHz:                     800.0000
BogoMIPS:                        4224.00
Virtualization:                  VT-x
L1d cache:                       384 KiB
L1i cache:                       256 KiB
L2 cache:                        16 MiB
NUMA node0 CPU(s):               

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [42]:
# Run it "with" the above code.
! cd ~/courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_intel 16384 16384  >& intel_with_loop.perf; grep 'D   refs\|D1' intel_with_loop.perf

==3644457== D   refs:      2,326,894  (1,650,798 rd   + 676,096 wr)
==3644457== D1  misses:       23,694  (   10,735 rd   +  12,959 wr)
==3644457== D1  miss rate:       1.0% (      0.7%     +     1.9%  )


In [43]:
# Let's do some math here
total_number_of_accesses_before_the_loop =2285928
total_number_of_accesses_after_the_loop = 2326894
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13447
total_number_of_misses_after_the_loop = 23694
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 40966 misses in the loop 10247 miss_rate 0.250134
